Madoria-EDA

My analysis is on the gross profit in the domestic vs foreign box office market and the production budget. My analysis utilizes the movie_gross, movie_budgets, and tmbd_movies charts:

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

After importing, pandas and matplotlib, I decided to get a peek of the data including a description of each of the table values and begin cleaning. Starting with the bom.movie_gross data:

In [ ]:
df1= pd.read_csv('data/bom.movie_gross.csv')
df1.head()

In [ ]:
df1.info()

In [ ]:
gross= df1[df1['studio'].notna() & df1['domestic_gross'].notna()]
gross

In [ ]:
gross['foreign_gross']= gross['foreign_gross'].fillna(0)

In [ ]:
gross['foreign_gross'] = gross['foreign_gross'].str.replace(',', '').astype(float)

In [ ]:
gross.info()

In [ ]:
top_10_studios_domestic= gross.groupby('studio')['domestic_gross'].sum().sort_values(ascending=False).head(10)
top_10_studios_domestic

In [ ]:
top_10_studios_foreign= gross.groupby('studio')['foreign_gross'].sum().sort_values(ascending=False).head(10)
top_10_studios_foreign

Now I move on to tn.movie budgets and do some more cleaning:

In [ ]:
df2= pd.read_csv('data/tn.movie_budgets.csv')
df2.head()

In [ ]:
df2.info()

In [ ]:
cols = ['production_budget', 'domestic_gross', 'worldwide_gross']
df2[cols] = df2[cols].replace('[\$,]','', regex=True).astype(float)
df2

In [ ]:
"""I was going to do percent, but I believe this is still understandable"""

df2['dom_profit_margin']= (df2['domestic_gross'] - df2['production_budget']) / df2['domestic_gross']
df2['ww_profit_margin']= (df2['worldwide_gross'] - df2['production_budget'])/ df2['worldwide_gross']
df2

In [ ]:
top_20_movies_worldwide= df2.nlargest(20, 'worldwide_gross')['movie'].reset_index()
top_20_movies_worldwide 

In [ ]:
top_world_profit = df2.sort_values('worldwide_gross', ascending=False).iloc[:20]

fig, ax= plt.subplots(figsize=(14,8))

ax.scatter(top_world_profit['production_budget'], top_world_profit['worldwide_gross'], s=100)
ax.set_title("Worldwide Gross by Budget")
ax.set_xlabel("Production Budget (in millions)")
ax.set_ylabel("Worldwide Gross (in billions)")

In [ ]:
top_10_movies_domestic= df2.nlargest(20, 'domestic_gross')['movie'].reset_index()
top_10_movies_domestic

In [ ]:
top_dom_profit = df2.sort_values('domestic_gross', ascending=False).iloc[:20]

fig, ax= plt.subplots(figsize=(14,8))

ax.scatter(top_dom_profit['production_budget'], top_dom_profit['domestic_gross'], c='r', s=100)
ax.set_title("Domestic Gross by Budget")
ax.set_xlabel("Production Budget (in 100 millions)")
ax.set_ylabel("Domestic Gross (in 100 millions)")

Based on the sample data I have, we've gotten some insight from the other two dataframes that:

 -certain studios seem to gross more than others whether foreign or domestic
 
 -there's a small correlation between the production budget and the overall gross

Now I'm moving on to the very last dataframe which is tmdb.movies just to see if anything ties into that:

In [ ]:
df3= pd.read_csv('data/tmdb.movies.csv')
df3.head()

In [ ]:
df3.info()

In [ ]:
df3.describe()

In [ ]:
df3['vote_high_low']= ["High Votes" if x > 194 else "Low Votes" for x in df3['vote_count']]
df3.head()

In [ ]:
most_popular= df3.nlargest(20, 'popularity')['genre_ids'].sort_values()
most_popular

I can see here that certain genres do seem to appear more in the top 20 list, but I need more evidence to confirm. 

Combining Tables:

Here I want to combine the tables to get more insight on studio profits and hopefully have more insight on the genres. 

First to combine, the movie gross and movie budget dataframes:

In [ ]:
new_movie= df1.merge(df2, left_on=df1['title'], right_on=df2['movie']).reset_index(drop=True)
new_movie

In [ ]:
mega_movie= mega_movie.drop_duplicates(subset=['movie'], keep='first')

In [ ]:
avg_studio= new_movie.groupby('studio').mean().reset_index()
avg_studio= avg_studio[avg_studio['dom_profit_margin'] > 0]
avg_studio

In [ ]:
avg_studio = avg_studio.sort_values('dom_profit_margin', ascending=False).iloc[:10]

fig, ax= plt.subplots(figsize=(14,8))

ax.bar(avg_studio['studio'], avg_studio['dom_profit_margin'])
ax.set_title= 'Top Studios'
ax.set_xlabel= 'Studios'
ax.set_ylabel= 'Domestic Profit Margin'

In [ ]:
avg_studio = avg_studio.sort_values('ww_profit_margin', ascending=False).iloc[:10]

fig, ax= plt.subplots(figsize=(15,8))
sns.barplot(data=avg_studio, x='studio', y='ww_profit_margin')

This new studio information definitely differs from the lists up top. I'm going to continue to make sure it's not a fluke. 

Now I'll combine the combined dataframe I made just now with the very last movies dataframe. 

In [ ]:
"""Had to do this first to merge again"""
del new_movie['key_0']

In [ ]:
mega_movie= new_movie.merge(df3, left_on=new_movie['title'], right_on=df3['title']).reset_index(drop=True)
mega_movie

In [ ]:
mega_movie.info()

There's dupes and nulls. Oh my

In [ ]:
mega_dupes= mega_movie[mega_movie.duplicated(['movie'])]
mega_dupes

In [ ]:
mega_movie= mega_movie.drop_duplicates(subset=['movie'], keep='first')

In [ ]:
mega_movie.drop(['key_0', 'title_x', 'id_x', 'release_date_x', 'id_y', 'release_date_y'], axis=1)

In [ ]:
mega_studio= mega_movie.groupby('studio').mean().reset_index()
mega_studio= mega_studio[mega_studio['dom_profit_margin'] > 0]
mega_studio

In [ ]:
mega_studio = mega_studio.sort_values('dom_profit_margin', ascending=False).iloc[:10]

fig, ax= plt.subplots(figsize=(14,8))

ax.bar(mega_studio['studio'], mega_studio['dom_profit_margin'])
ax.set_title= 'Top Studios'
ax.set_xlabel= 'Studios'
ax.set_ylabel= 'Domestic Profit Margin'

In [ ]:
mega_studio = mega_studio.sort_values('ww_profit_margin', ascending=False).iloc[:10]

fig, ax= plt.subplots(figsize=(14,8))

ax.bar(mega_studio['studio'], mega_studio['ww_profit_margin'])
ax.set_title= 'Top Studios'
ax.set_xlabel= 'Studios'
ax.set_ylabel= 'Worldwide Profit Margin'

In [ ]:
type(mega_movie['genre_ids'][0])

In [ ]:
def to_1D(series):
    return pd.Series([x for genres in series for x in genres])

In [ ]:
genre_list= to_1D(mega_movie['genre_ids']).value_counts()
genre_list

In [ ]:
fig, ax= plt.subplots()

to_1D(mega_movie['genre_ids']).value_counts()[:10].plot(kind='barh')
ax.set_xlabel("Frequency")
ax.set_ylabel("Genre")
ax.set_title("Movie Genres by Frequency")

So now we are at the end of my journal. There are certain studios that have a higher gross, but that doesn't really translate to the overall profit margin. And certain genres definitely dominate the box office. 

More analysis would be needed on the names of the genre ids, the types of movies associated with the Top Studios by Profit Margin, more timeline data to find possible trends in in genres or profit margin, and more. 
